In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from keras.models import Sequential, Model
from keras.layers import Dense, Input
from keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import tensorflow as tf
import numpy as np

from tensorflow.keras.layers import Flatten,Dense,Input,Reshape
from tensorflow.keras.models import Sequential
from random import randrange

### Import dataset

In [2]:
df = pd.read_csv('../Data/processed_df.csv')

In [3]:
df.head()

,handgrip,weight_loss,exaustion,poor_endurance,gait_speed,balance,hearing_deficit,distance_vision,near_vision,depression_scale,...,idade_morador_8,idade_morador_9,idade_morador_10,idade_morador_11,idade_morador_12,idade_morador_13,idade_morador_14,adl_index,aadl_index,iadl_index
0,24.333333,0.0,1.0,1.0,3.500,4.0,2.0,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,7.0,8.0
1,25.666667,0.0,1.0,1.0,3.300,4.0,4.0,3.0,3.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,4.0,8.0
2,14.666667,NaN,2.0,1.0,4.595,4.0,2.0,3.0,4.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,7.0,8.0
3,24.000000,0.0,1.0,1.0,5.610,4.0,3.0,2.0,2.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,5.0,8.0
4,NaN,0.0,4.0,2.0,38.145,NaN,3.0,2.0,3.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,3.0,3.0


In [4]:
df.columns

Index(['handgrip', 'weight_loss', 'exaustion', 'poor_endurance', 'gait_speed',
       'balance', 'hearing_deficit', 'distance_vision', 'near_vision',
       'depression_scale', 'sleep_quality', 'temporal_orientation',
       'memory_recall', 'semantic_memory', 'verbal_fluency', 'sexo', 'regiao',
       'etinia', 'sexo_morador_1', 'sexo_morador_2', 'sexo_morador_3',
       'sexo_morador_4', 'sexo_morador_5', 'sexo_morador_6', 'sexo_morador_7',
       'sexo_morador_8', 'sexo_morador_9', 'sexo_morador_10',
       'sexo_morador_11', 'sexo_morador_12', 'sexo_morador_13',
       'sexo_morador_14', 'papel_morador_1', 'papel_morador_2',
       'papel_morador_3', 'papel_morador_4', 'papel_morador_5',
       'papel_morador_6', 'papel_morador_7', 'papel_morador_8',
       'papel_morador_9', 'papel_morador_10', 'papel_morador_11',
       'papel_morador_12', 'papel_morador_13', 'papel_morador_14',
       'idade_morador_1', 'idade_morador_2', 'idade_morador_3',
       'idade_morador_4', 'idade_morad

### Remove missing data

In [5]:
ci_map = {
    'cognitive': ['semantic_memory', 'verbal_fluency', 'memory_recall', 'temporal_orientation'],
    'psychologicao': ['depression_scale', 'sleep_quality'],
    'locomotor': ['gait_speed', 'balance'],
    'vitality': ['handgrip', 'poor_endurance', 'weight_loss', 'exaustion'],
    'sensory': ['distance_vision', 'near_vision', 'hearing_deficit']
}

# ci_map = {
#     'cognitive': ['semantic_memory', 'verbal_fluency', 'memory_recall'],
#     'psychologicao': ['depression_scale', 'sleep_quality'],
#     'locomotor': ['gait_speed', 'balance'],
#     'vitality': ['handgrip', 'poor_endurance', 'weight_loss', 'exaustion'],
#     'sensory': ['distance_vision', 'near_vision', 'hearing_deficit']
# }
ci_cols = []
[ci_cols.extend(value) for value in ci_map.values()]

ci_cols

['semantic_memory',
 'verbal_fluency',
 'memory_recall',
 'temporal_orientation',
 'depression_scale',
 'sleep_quality',
 'gait_speed',
 'balance',
 'handgrip',
 'poor_endurance',
 'weight_loss',
 'exaustion',
 'distance_vision',
 'near_vision',
 'hearing_deficit']

In [6]:
df_clean = df.dropna(subset=ci_cols)
df_clean.shape

(6955, 63)

### Split Train and test


In [7]:
import pickle
# Define dataset
df_autoencoder = df_clean[ci_cols]
# Split the data into training and testing sets
X_train, X_test = train_test_split(df_autoencoder, test_size=0.2, random_state=42)
X_train, X_val = train_test_split(X_train, test_size=0.25, random_state=42)


scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

X_train_scaled.shape, X_val_scaled.shape, X_test_scaled.shape


with open('./out/autoencoder_scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)


### Import Genetic Result

In [12]:
model = tf.keras.models.load_model('./out/best_model.h5')

In [14]:
model.predict(X_test_scaled)

44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


array([[-0.8549272 , -0.58710706, -0.90015924, ...,  0.9048408 ,
         0.8758171 ,  0.50359416],
       [ 0.06441069,  0.21373898,  0.23768081, ..., -0.51436234,
        -0.48540825, -0.37488878],
       [-1.3360432 , -1.2248752 , -1.3158071 , ...,  0.830806  ,
         0.6561135 ,  0.59718376],
       ...,
       [ 0.43996298,  0.02689853, -0.32442033, ..., -0.37698272,
        -0.1017884 , -0.05356403],
       [ 0.6770048 ,  1.1895038 ,  1.3497763 , ..., -0.63882494,
        -0.6396715 , -0.6309602 ],
       [ 0.06028272,  0.042566  ,  0.21045123, ..., -0.23319641,
        -0.2776718 , -0.09538497]], dtype=float32)

In [15]:
from sklearn.metrics import r2_score

decoded_data = model.predict(X_test_scaled)
global_r2 = r2_score(X_test_scaled, decoded_data)
print(global_r2)

44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
0.3801153006514832
